In [1]:
import os, glob
import re
import numpy as np

In [2]:
out_dir = "../data/maynard_nested_xval/"

lr_pattern = re.compile("Learning Rate: (\d+.\d+(e-\d+)*)")
alpha_pattern = re.compile("Alpha: (\d+.*\d*)")
acc_pattern = re.compile(".* Acc: (\d+.\d+)")

folds = 6
sub_folds = folds-1
repetitions = 5

# Store lr and alpha that lead to best val acc. for each fold (as well as the acc.)
best_lrs = np.zeros((folds, sub_folds))
best_alphas = np.zeros((folds, sub_folds))
best_val_accs = np.zeros((folds, sub_folds))

for i in range(150):
    pth = os.path.join(out_dir, "gnhex_maynard_xval.672174_%d.out" % i)
    
    test_fold = i // (sub_folds * repetitions)
    tmp = i - (test_fold * sub_folds * repetitions)
    val_fold = tmp // repetitions
    
    if os.path.exists(pth):
        train_acc, val_acc = [],[]
        
        for line in open(pth, "r"):
            if line.startswith("Learning Rate:"):
                lr = float(lr_pattern.match(line).groups()[0])
            if line.startswith("Alpha:"):
                alpha = float(alpha_pattern.match(line).groups()[0])
                
            if line.startswith("val Loss"):
                acc = float(acc_pattern.match(line).groups()[0])
                val_acc.append(acc)
            if line.startswith("train Loss"):
                acc = float(acc_pattern.match(line).groups()[0])
                train_acc.append(acc)
        
        if len(val_acc) == 0:
            continue
                
        # Find best-performing model:
        best_ind = np.argmax(np.array(val_acc))
        
        if val_acc[best_ind] > best_val_accs[test_fold, val_fold]:
            best_val_accs[test_fold, val_fold] = val_acc[best_ind]
            best_lrs[test_fold, val_fold] = lr
            best_alphas[test_fold, val_fold] = alpha

In [3]:
print(best_val_accs)
print(best_lrs)
print(best_alphas)

[[0.3273 0.4647 0.6158 0.6281 0.7183]
 [0.3375 0.5188 0.5819 0.5947 0.7026]
 [0.5183 0.5559 0.3845 0.5646 0.726 ]
 [0.5491 0.5447 0.3247 0.602  0.7285]
 [0.537  0.5704 0.5799 0.544  0.4914]
 [0.5661 0.5318 0.5744 0.538  0.4266]]
[[0.0001118 0.0006247 0.0008457 0.0009427 0.0007056]
 [0.0009532 0.0005031 0.0008974 0.0007403 0.0008071]
 [0.0001152 0.0007056 0.0007925 0.0005237 0.0003473]
 [0.0009529 0.0007651 0.0005434 0.0009506 0.0003573]
 [0.0005617 0.000763  0.0001072 0.0007509 0.0007165]
 [0.0009475 0.0006576 0.0006036 0.0002143 0.0002234]]
[[0.06566 0.08676 0.09531 0.06213 0.07951]
 [0.058   0.09421 0.08833 0.08537 0.02993]
 [0.07102 0.07245 0.07126 0.08979 0.06747]
 [0.08672 0.07126 0.06279 0.06364 0.09957]
 [0.06327 0.06248 0.09744 0.01142 0.0683 ]
 [0.08156 0.07579 0.03896 0.01768 0.05769]]


In [4]:
# Find associated model files
for i in range(folds):
    for j in range(sub_folds):
        lr = best_lrs[i,j]
        alpha = best_alphas[i,j]
        
        model_file = "../data/gnethex_maynard_%d_%d_lr%.4g_alpha%.4g.pth" % (i,j,lr,alpha)
        
        if os.path.exists(model_file):
            print(model_file)

../data/gnethex_maynard_0_0_lr0.0001118_alpha0.06566.pth
../data/gnethex_maynard_0_1_lr0.0006247_alpha0.08676.pth
../data/gnethex_maynard_0_2_lr0.0008457_alpha0.09531.pth
../data/gnethex_maynard_0_3_lr0.0009427_alpha0.06213.pth
../data/gnethex_maynard_0_4_lr0.0007056_alpha0.07951.pth
../data/gnethex_maynard_1_0_lr0.0009532_alpha0.058.pth
../data/gnethex_maynard_1_1_lr0.0005031_alpha0.09421.pth
../data/gnethex_maynard_1_2_lr0.0008974_alpha0.08833.pth
../data/gnethex_maynard_1_3_lr0.0007403_alpha0.08537.pth
../data/gnethex_maynard_1_4_lr0.0008071_alpha0.02993.pth
../data/gnethex_maynard_2_0_lr0.0001152_alpha0.07102.pth
../data/gnethex_maynard_2_1_lr0.0007056_alpha0.07245.pth
../data/gnethex_maynard_2_2_lr0.0007925_alpha0.07126.pth
../data/gnethex_maynard_2_3_lr0.0005237_alpha0.08979.pth
../data/gnethex_maynard_2_4_lr0.0003473_alpha0.06747.pth
../data/gnethex_maynard_3_0_lr0.0009529_alpha0.08672.pth
../data/gnethex_maynard_3_1_lr0.0007651_alpha0.07126.pth
../data/gnethex_maynard_3_2_lr0.0